<a href="https://colab.research.google.com/github/HamzaBahsir/NLP/blob/main/Keyword_and_Semantic_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import math
from scipy import spatial


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install sentence_transformers openai unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 11.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=cef68adcf6f9dcc4148835fd3fa40c77a462e8473adf32f1b7c8091763fe2b01
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a

Let's discuss why we need chunking/ need to divide data - context length

In [3]:
txt = '''I would like to get your all  thoughts on the bond yield increase this week.  I am not worried about the market downturn but the sudden increase in yields. On 2/16 the 10 year bonds yields increased by almost  9 percent and on 2/19 the yield increased by almost 5 percent.

Key Points from the CNBC Article:

* **The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.**
* **Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic.**
* **However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.**

The recent rise in bond yields and U.S. inflation expectations has some investors wary that a repeat of the 2013 “taper tantrum” could be on the horizon.

The benchmark U.S. 10-year Treasury note climbed above 1.3% for the first time since February 2020 earlier this week, while the 30-year bond also hit its highest level for a year. Yields move inversely to bond prices.

Yields tend to rise in lockstep with inflation expectations, which have reached their highest levels in a decade in the U.S., powered by increased prospects of a large fiscal stimulus package, progress on vaccine rollouts and pent-up consumer demand.

The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.

Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. The Fed and others have maintained supportive tones in recent policy meetings, vowing to keep financial conditions loose as the global economy looks to emerge from the Covid-19 pandemic.

However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.

With central bank support removed, bonds usually fall in price which sends yields higher. This can also spill over into stock markets as higher interest rates means more debt servicing for firms, causing traders to reassess the investing environment.

“The supportive stance from policymakers will likely remain in place until the vaccines have paved a way to some return to normality,” said Shane Balkham, chief investment officer at Beaufort Investment, in a research note this week.

“However, there will be a risk of another ‘taper tantrum’ similar to the one we witnessed in 2013, and this is our main focus for 2021,” Balkham projected, should policymakers begin to unwind this stimulus.

Long-term bond yields in Japan and Europe followed U.S. Treasurys higher toward the end of the week as bondholders shifted their portfolios.

“The fear is that these assets are priced to perfection when the ECB and Fed might eventually taper,” said Sebastien Galy, senior macro strategist at Nordea Asset Management, in a research note entitled “Little taper tantrum.”

“The odds of tapering are helped in the United States by better retail sales after four months of disappointment and the expectation of large issuance from the $1.9 trillion fiscal package.”

Galy suggested the Fed would likely extend the duration on its asset purchases, moderating the upward momentum in inflation.

“Equity markets have reacted negatively to higher yield as it offers an alternative to the dividend yield and a higher discount to long-term cash flows, making them focus more on medium-term growth such as cyclicals” he said. Cyclicals are stocks whose performance tends to align with economic cycles.

Galy expects this process to be more marked in the second half of the year when economic growth picks up, increasing the potential for tapering.

## Tapering in the U.S., but not Europe

Allianz CEO Oliver Bäte told CNBC on Friday that there was a geographical divergence in how the German insurer is thinking about the prospect of interest rate hikes.

“One is Europe, where we continue to have financial repression, where the ECB continues to buy up to the max in order to minimize spreads between the north and the south — the strong balance sheets and the weak ones — and at some point somebody will have to pay the price for that, but in the short term I don’t see any spike in interest rates,” Bäte said, adding that the situation is different stateside.

“Because of the massive programs that have happened, the stimulus that is happening, the dollar being the world’s reserve currency, there is clearly a trend to stoke inflation and it is going to come. Again, I don’t know when and how, but the interest rates have been steepening and they should be steepening further.”

## Rising yields a ‘normal feature’

However, not all analysts are convinced that the rise in bond yields is material for markets. In a note Friday, Barclays Head of European Equity Strategy Emmanuel Cau suggested that rising bond yields were overdue, as they had been lagging the improving macroeconomic outlook for the second half of 2021, and said they were a “normal feature” of economic recovery.

“With the key drivers of inflation pointing up, the prospect of even more fiscal stimulus in the U.S. and pent up demand propelled by high excess savings, it seems right for bond yields to catch-up with other more advanced reflation trades,” Cau said, adding that central banks remain “firmly on hold” given the balance of risks.

He argued that the steepening yield curve is “typical at the early stages of the cycle,” and that so long as vaccine rollouts are successful, growth continues to tick upward and central banks remain cautious, reflationary moves across asset classes look “justified” and equities should be able to withstand higher rates.

“Of course, after the strong move of the last few weeks, equities could mark a pause as many sectors that have rallied with yields look overbought, like commodities and banks,” Cau said.

“But at this stage, we think rising yields are more a confirmation of the equity bull market than a threat, so dips should continue to be bought'''

In [4]:
len(txt)

6422

In [5]:
from transformers import BertTokenizer, BertModel


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
# text = "This is a long text that needs to be encoded and decoded."
encoded_text = tokenizer.encode(txt, padding=True, truncation=True, return_tensors='pt')

In [7]:
encoded_text.shape

torch.Size([1, 512])

In [8]:
decoded_text = tokenizer.decode(encoded_text[0], skip_special_tokens=True)

In [9]:
decoded_text

'i would like to get your all thoughts on the bond yield increase this week. i am not worried about the market downturn but the sudden increase in yields. on 2 / 16 the 10 year bonds yields increased by almost 9 percent and on 2 / 19 the yield increased by almost 5 percent. key points from the cnbc article : * * * the “ taper tantrum ” in 2013 was a sudden spike in treasury yields due to market panic after the federal reserve announced that it would begin tapering its quantitative easing program. * * * * * major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. * * * * * however, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation. * * the recent rise in bond yields and u. s. inflation expectations has some investors wary that a re

In [10]:
len(decoded_text)

2542

Let's Keyword Search using TF-IDF approach


# TF-IDF: A Simple Explanation

## What is TF-IDF?

TF-IDF stands for "Term Frequency-Inverse Document Frequency." It's a way to figure out how important a word is in a document or a collection of documents.

## Breaking it Down

### TF (Term Frequency)

- This measures how often a word appears in a document.
- The more a word appears, the more important it might be.

**Example:**
In the sentence "The cat sat on the mat," the word "the" appears twice, while "cat" appears once.

### IDF (Inverse Document Frequency)

- This measures how common or rare a word is across all documents.
- Rare words are often more important or informative.

**Example:**
If you have 1000 documents about animals, and "cat" appears in 100 of them, while "aardvark" appears in only 5, "aardvark" would have a higher IDF.

## Putting TF and IDF Together

TF-IDF combines these two measures:
- It gives a high score to words that appear frequently in a document but are rare across all documents.
- It gives a low score to words that appear in many documents or appear rarely in a document.

## Real-World Example

Imagine you have a collection of news articles:

1. An article about a football match might have high TF-IDF scores for words like "goal," "referee," and player names.
2. An article about cooking might have high TF-IDF scores for words like "recipe," "ingredients," and specific dish names.

Common words like "the," "and," or "is" would have low TF-IDF scores in both articles because they appear frequently in almost all documents.

## Why is TF-IDF Useful?

1. **Search Engines**: It helps rank search results by identifying the most relevant documents.
2. **Content Recommendation**: It can find similar articles or documents.
3. **Text Summarization**: It can identify the most important words or sentences in a document.
4. **Spam Detection**: It can help identify unusual word patterns that might indicate spam.

## Conclusion

TF-IDF is a simple but powerful tool that helps us understand the importance of words in documents. It balances how often a word appears in a document with how unique it is across all documents, giving us valuable insights into text data.

In [11]:
nltk.download('punkt_tab')
# List of documents
documents = [
    "The cat is playing in the garden",
    "A dog and cat are good pets",
    "Cats love to chase mice",
    "machine learning is based on algorithms ",
    "Deep learning uses neural networks",
    "Recurrent networks have connections",
    "The cost of this shirt is $15"
]

# Tokenize documents into words

tokenized_docs = [word_tokenize(d) for d in documents]

# Inverted index mapping words to document IDs
index = {}
for i, doc in enumerate(tokenized_docs):
    for word in doc:
        if word not in index:
            index[word] = []
        index[word].append(i)

print(f'Inverted Index:\n {index}')



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Inverted Index:
 {'The': [0, 6], 'cat': [0, 1], 'is': [0, 3, 6], 'playing': [0], 'in': [0], 'the': [0], 'garden': [0], 'A': [1], 'dog': [1], 'and': [1], 'are': [1], 'good': [1], 'pets': [1], 'Cats': [2], 'love': [2], 'to': [2], 'chase': [2], 'mice': [2], 'machine': [3], 'learning': [3, 4], 'based': [3], 'on': [3], 'algorithms': [3], 'Deep': [4], 'uses': [4], 'neural': [4], 'networks': [4, 5], 'Recurrent': [5], 'have': [5], 'connections': [5], 'cost': [6], 'of': [6], 'this': [6], 'shirt': [6], '$': [6], '15': [6]}


In [12]:

# Compute TF-IDF weights for words
def tfidf(word, id):
    # Term frequency - no. of times word appears in document
    tf = tokenized_docs[id].count(word)

    # No. of documents containing the word
    df = len(index[word]) if word in index else 0

    # Inverse document frequency
    idf =  math.log(len(documents) / (df+1))

    return tf * idf



In [13]:
# Search query
query = "$15"
query_words = word_tokenize(query)

In [14]:
scores = {id:0 for id in range(len(documents))}
for q in query_words:
    if q in index:
        for id in index[q]:
            scores[id] += tfidf(q, id)

scores

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 2.505525936990736}

In [15]:

# Rank documents
ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)

# Return ranked documents
for id, score in ranked:
    print(f"Document {id}: {documents[id]}")
    print(f"Score: {score}")

Document 6: The cost of this shirt is $15
Score: 2.505525936990736
Document 0: The cat is playing in the garden
Score: 0
Document 1: A dog and cat are good pets
Score: 0
Document 2: Cats love to chase mice
Score: 0
Document 3: machine learning is based on algorithms 
Score: 0
Document 4: Deep learning uses neural networks
Score: 0
Document 5: Recurrent networks have connections
Score: 0


## Semantic Search

In [16]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
import pandas as pd


In [17]:
documents = [
    "The cat is playing in the garden",
    "A dog and cat are good pets",
    "Cats love to chase mice",
    "Machine learning is based on algorithms",
    "Deep learning uses neural networks",
    "Recurrent networks have connections",
     "The cost of this shirt is $15"
]


In [18]:
model = SentenceTransformer('all-MiniLM-L6-v2').to('cuda') #cpu


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [20]:
embeddings = model.encode(documents,convert_to_tensor=True).to('cpu')

In [21]:
embeddings.shape

torch.Size([7, 384])

In [22]:
embeddings

tensor([[ 0.0985, -0.0455,  0.0097,  ...,  0.0113,  0.0335, -0.0079],
        [ 0.0143, -0.0200,  0.0689,  ...,  0.0864,  0.0669,  0.0567],
        [ 0.0305, -0.0297,  0.0800,  ...,  0.1213,  0.0618,  0.0443],
        ...,
        [-0.1011, -0.0130,  0.0732,  ...,  0.0614,  0.0381, -0.0397],
        [-0.0761, -0.0688,  0.0270,  ...,  0.1124,  0.0479, -0.0509],
        [-0.0226,  0.0855,  0.0442,  ..., -0.2102, -0.0619,  0.0028]])

In [23]:
# User query
query = "furry animals"

# Encode query
query_embedding = model.encode(query)

In [24]:
query_embedding.shape

(384,)

In [25]:
scores = [spatial.distance.cosine(query_embedding, doc) for doc in embeddings]


In [26]:
scores

[np.float32(0.84655595),
 np.float32(0.4998132),
 np.float32(0.56891125),
 np.float32(0.99278724),
 np.float32(0.9309803),
 np.float32(0.9278096),
 np.float32(0.9691598)]

In [27]:
scores = [scipy.spatial.distance.cosine(query_embedding, doc) for doc in embeddings]

# Print each score
for i, doc in enumerate(documents):
    print(f"Document {i}: {documents[i]}")
    print(f"Score: {1 - scores[i]:.4f}")

print("Most similar document:", documents[scores.index(min(scores))])

Document 0: The cat is playing in the garden
Score: 0.1534
Document 1: A dog and cat are good pets
Score: 0.5002
Document 2: Cats love to chase mice
Score: 0.4311
Document 3: Machine learning is based on algorithms
Score: 0.0072
Document 4: Deep learning uses neural networks
Score: 0.0690
Document 5: Recurrent networks have connections
Score: 0.0722
Document 6: The cost of this shirt is $15
Score: 0.0308
Most similar document: A dog and cat are good pets


In [28]:
def semantic_search(query,embeddings, documents):
    query_embedding = model.encode(query)

    document_embeddings = embeddings
    scores = [scipy.spatial.distance.cosine(query_embedding, doc) for doc in document_embeddings]

    # Print scores
    for i, doc in enumerate(documents):
        print(f"{doc}: {1 - scores[i]:.4f}")

    most_similar_doc = documents[scores.index(min(scores))]
    print("Most similar document:", most_similar_doc)


In [29]:
semantic_search("Furry animals",embeddings,documents)

The cat is playing in the garden: 0.1534
A dog and cat are good pets: 0.5002
Cats love to chase mice: 0.4311
Machine learning is based on algorithms: 0.0072
Deep learning uses neural networks: 0.0690
Recurrent networks have connections: 0.0722
The cost of this shirt is $15: 0.0308
Most similar document: A dog and cat are good pets


In [30]:
semantic_search("The cost of this shirt is $1235467",embeddings,documents)

The cat is playing in the garden: -0.0858
A dog and cat are good pets: -0.0850
Cats love to chase mice: -0.0521
Machine learning is based on algorithms: 0.0389
Deep learning uses neural networks: 0.0534
Recurrent networks have connections: -0.0189
The cost of this shirt is $15: 0.8588
Most similar document: The cost of this shirt is $15
